In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [15]:
names = ['API','Surf_X','Surf_Y','Date_Drilling','Date_Completion','Date_Production','Lateral_Length','Depth_TVD_PPLS','Erosion_PPLS','Pressure_PPLS','TOC_PPLS','Vcarb_PPLS','Vsand_PPLS','Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

df = pd.read_csv('./TrainSample.csv', 
                         header = None, sep = ';', decimal = ',', names = names, skiprows = 1)
df.set_index('API')

df.head()

,API,Surf_X,Surf_Y,Date_Drilling,Date_Completion,Date_Production,Lateral_Length,Depth_TVD_PPLS,Erosion_PPLS,Pressure_PPLS,...,Min_Rate_Slurry,ShutInPressure_Fil,ShutInPressure_Initial,ISIP,Shot_Density,Shot_Total,Proppant_per_ft,Stage_Spacing,GasCum360,OilCum360
0,661,0.429600,-0.193201,01/06/2013,21/03/2014,08/05/2014,-1.124200,-0.214336,-1.262296,-0.153197,...,0.635528,-0.067825,0.069978,-0.253698,-1.253690,0.181738,0.979100,-0.184434,-0.323593,-0.659768
1,523,-0.082672,-0.539854,31/08/2014,17/02/2015,03/08/2015,1.286521,0.870034,0.717834,0.674376,...,-1.364776,0.835789,1.136126,0.995649,0.667339,-0.560750,1.999634,-1.225036,-0.356343,1.782547
2,703,-0.457415,-0.082104,04/11/2013,08/07/2014,30/07/2014,-0.781945,0.511093,0.592495,-0.937430,...,0.667127,-0.287786,0.979602,-0.386425,-1.253690,0.181738,-0.492791,-0.181300,-1.067145,0.283035
3,473,0.376282,-0.310432,24/11/2011,11/05/2012,13/07/2012,-0.442801,0.025900,-1.093255,0.164815,...,0.033723,-1.034051,-1.342103,-1.546778,-0.293176,-1.303237,-0.295062,2.135885,-0.734454,-0.328650
4,309,-0.113110,-0.111412,20/11/2014,07/04/2015,23/07/2015,0.258511,0.190913,1.229474,-0.347126,...,-0.868364,0.923707,1.470667,0.561383,0.667339,-0.560750,2.034630,-1.226546,-0.271119,0.975603


In [29]:
print(len(df.columns))

47


In [30]:
for i in range(47):
    print(len(df[[i]]))

460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460


In [33]:
print(df[[44]])

     Stage_Spacing
0        -0.184434
1        -1.225036
2        -0.181300
3         2.135885
4        -1.226546
5              NaN
6        -0.213844
7        -1.232964
8        -0.149418
9        -0.212425
10             NaN
11             NaN
12             NaN
13        2.032007
14       -0.178793
15       -1.234000
16       -0.162951
17       -1.216550
18       -0.184966
19        1.880874
20             NaN
21       -1.238962
22       -0.286218
23        0.574318
24             NaN
25        1.984326
26        0.107721
27       -0.186314
28       -1.302854
29       -0.308990
..             ...
430      -1.263277
431            NaN
432      -0.217250
433      -1.234549
434      -0.270324
435       1.422366
436      -1.243689
437       0.278012
438       1.593895
439      -0.194072
440       0.045459
441      -0.219886
442      -1.166170
443      -0.182746
444      -0.156229
445      -1.249096
446      -0.025318
447      -1.227169
448      -0.161764
449      -1.237432
450      -0.

In [9]:
cols = ['Date_Drilling','Date_Completion','Date_Production','Pressure_PPLS','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing']

#'Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

#df.drop(df.columns[cols], axis=1, inplace=True)
df.drop(cols, 1, inplace=True)

In [10]:
df.describe()

C:\Users\COCHE\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,API,Surf_X,Surf_Y,Lateral_Length,Depth_TVD_PPLS,Erosion_PPLS,TOC_PPLS,Vcarb_PPLS,Vsand_PPLS,Vclay_PPLS,...,Proppant_Designed,Proppant_in_Formation,Avg_Breakdown_Pressure,Avg_Treating_Pressure,Max_Treating_pressure,Min_Treating_Pressure,Avg_Rate_Slurry,Max_Rate_Slurry,GasCum360,OilCum360
count,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,...,363.000000,362.000000,361.000000,363.000000,362.000000,362.000000,363.000000,361.000000,460.000000,460.000000
mean,408.271739,0.048312,-0.020504,-0.038670,-0.020932,-0.044322,0.027042,0.009838,-0.018139,-0.016165,...,-0.032791,-0.030408,-0.006015,-0.029595,-0.073869,-0.045175,0.035797,0.048529,-0.004878,-0.011122
std,228.391358,0.972776,0.974876,0.982085,0.947728,0.992127,0.995567,1.032273,1.026098,1.016012,...,0.999613,0.998820,1.017355,0.994565,0.749684,1.083354,0.964642,0.991292,1.015833,0.986570
min,1.000000,-2.426259,-2.740075,-2.325205,-2.958958,-2.135948,-2.169983,-3.404962,-2.125431,-1.957799,...,-1.509136,-1.519976,-2.741104,-4.641543,-3.833633,-7.938938,-2.877936,-3.264990,-1.241844,-0.846812
25%,206.750000,-0.377085,-0.631271,-0.723762,-0.675256,-0.854066,-0.593362,-0.294986,-0.718530,-0.569624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.670713,-0.762137
50%,412.000000,0.149286,-0.034792,-0.394263,-0.008885,-0.031200,0.324258,0.245879,-0.190943,-0.291989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.354850,-0.269251
75%,609.250000,0.511680,0.622794,0.578986,0.612669,0.845490,0.753375,0.786744,0.512507,0.263281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.443100,0.317948
max,793.000000,2.869421,3.143658,3.480066,3.885516,1.987123,1.860650,1.868475,3.326308,2.484361,...,3.209552,3.212776,2.968671,1.946600,1.652424,3.591486,3.813211,3.436429,5.366269,5.527111


In [ ]:
def correlation(data):
    correlation = data.corr()
    figure = plt.subplots( figsize =( 12 , 10 ) )
    color_map = sns.diverging_palette( 220 , 10 , as_cmap = True )
    figure = sns.heatmap(
        correlation, 
        cmap = color_map, 
        annot = True, 
        annot_kws = { 'fontsize' : 2 }
    )
    
correlation(data_train)

In [ ]:
from sklearn.decomposition import PCA 

principal_features = PCA(n_components = 10).fit_transform(data_train)

In [ ]:
"""
Rappport 1 :

Importation correcte des données
Début du travail exploratoire, affichage du head et description des données
Beaucoup de colonnes sont incomplètes => identification et élimination de ces colonnes
Observation des composantes corrélées.

Rapport 2 :

Elimination des colonnes incomplètes
Exploration des données de sorties
Choix de l'algorithme
"""